In [ ]:
# default_exp token_emb
# all_data

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Embeddings

## Imports

In [ ]:
from flair.data import Sentence

In [ ]:
from flair.embeddings import WordEmbeddings

In [ ]:
from flair.embeddings import TransformerWordEmbeddings

In [ ]:
from torch.nn import CosineSimilarity

### Load language model

In [ ]:
MODEL = TransformerWordEmbeddings('roberta-base')

## Pre-processing

### Class-less version

In [ ]:
LEX = 'bank'

In [ ]:
sents_strings = [
    'He found money at the bank.', 
    'She works at a bank.', 
    "The boat reached the river's bank."
]

In [ ]:
def proc_sentence(sent_string):
    sent = Sentence(sent_string)
    return sent

In [ ]:
sent_string = "I can't do this."
sent = proc_sentence(sent_string)

In [ ]:
assert len(sent) == 6

In [ ]:
for tok in sent:
    print(tok, tok.embedding)

In [ ]:
def embed_sentence(sent, MODEL):
    MODEL.embed(sent)
    return sent

In [ ]:
sent_emb = embed_sentence(sent, MODEL)

In [ ]:
for tok in sent_emb:
    print(tok, tok.embedding.shape)

In [ ]:
assert len(sent_emb) == 6

In [ ]:
assert sent_emb[0].embedding.shape[0] == 768

In [ ]:
def get_sent_toks(sent, lex):
    sent_toks = []
    for tok in sent:
        if tok.text == lex:
            sent_toks.append(tok)
    return sent_toks

In [ ]:
lex = 'cell'
sent = proc_sentence('A cell is a cell.')
sent_emb = embed_sentence(sent, MODEL)
sent_toks = get_sent_toks(sent_emb, lex)

In [ ]:
assert len(sent_toks) == 2

In [ ]:
def get_sent_vec(sent_toks, lex):
    sent_toks_vecs = [tok.embedding for tok in sent_toks]
    # return vector only for the first token in the sentence, ignore the rest; alternative: take mean of all target tokens
    sent_vec = sent_toks_vecs[0]
    return sent_vec

In [ ]:
assert get_sent_vec(sent_toks, lex).shape[0] == 768

In [ ]:
def get_cos_sim(vec_1, vec_2):
    cos_sim = CosineSimilarity(dim=0)
    sim_vecs = cos_sim(vec_1, vec_2)
    return float(sim_vecs)

In [ ]:
def get_sent_sim(sent_1, sent_2, lex):
    sent_vec_1 = get_sent_vec_
    cos_sim = get_cos_sim(sent_vec_1, sent_vec_2)

### Class version

#### Imports

In [ ]:
from fastcore.basics import patch

#### Variables

In [ ]:
lex = 'bank'

In [ ]:
sents_strings = [
    'He found money at the bank.', 
    'She works at a bank.', 
    "The boat reached the river's bank."
]

In [ ]:
sent_string = sents_strings[0]

#### Initialize `Sent` class

In [ ]:
class Sent():
    def __init__(self, string, model):
        self.string = string
        self.model = model

In [ ]:
sent = Sent(sent_string, MODEL)

In [ ]:
sent.string

'He found money at the bank.'

#### Tokenize sentence

In [ ]:
@patch(as_prop=True)
def sent(self: Sent):
    return Sentence(self.string)

In [ ]:
for tok in sent.sent:
    print(tok)

Token: 1 He
Token: 2 found
Token: 3 money
Token: 4 at
Token: 5 the
Token: 6 bank
Token: 7 .


In [ ]:
assert len(sent.sent) == 7

#### Embed sentence

In [ ]:
@patch
def get_sent_emb(self: Sent):
    sent_tmp = self.sent
    self.model.embed(sent_tmp)
    return sent_tmp

In [ ]:
sent = Sent(sent_string, MODEL)

In [ ]:
sent_emb = sent.get_sent_emb()

In [ ]:
for tok in sent_emb:
    print(tok, tok.embedding.shape)

Token: 1 He torch.Size([768])
Token: 2 found torch.Size([768])
Token: 3 money torch.Size([768])
Token: 4 at torch.Size([768])
Token: 5 the torch.Size([768])
Token: 6 bank torch.Size([768])
Token: 7 . torch.Size([768])


In [ ]:
assert len(sent_emb) == 7

In [ ]:
assert sent_emb[0].embedding.shape[0] == 768

#### Get target tokens

In [ ]:
def get_toks_tgt(sent, lex):
    sent_toks = []
    for tok in sent:
        if tok.text == lex:
            sent_toks.append(tok)
    return sent_toks

In [ ]:
sent_toks_tgt = get_toks_tgt(sent.sent, lex)

In [ ]:
assert len(sent_toks_tgt) == 1

#### Tokens

In [ ]:
sent_toks_tgt

[Token: 6 bank]

In [ ]:
sent.string

'He found money at the bank.'

In [ ]:
class Tok:
    def __init__(self, Token, Sent):
        self.text = Token.text
        self.context = Sent.string
        self.embedding = Token.embedding

In [ ]:
for tok in sent_emb.sent:
    tok = Tok(tok, sent)
    print(tok.text, tok.context, tok.embedding)

AttributeError: 'Sentence' object has no attribute 'sent'

In [ ]:
for tok in toks:
    print(tok, tok.embedding)

Token: 6 bank tensor([])


In [ ]:
def get_sent_vec(sent_toks, lex):
    sent_toks_vecs = [tok.embedding for tok in sent_toks]
    # return vector only for the first token in the sentence, ignore the rest; alternative: take mean of all target tokens
    sent_vec = sent_toks_vecs[0]
    return sent_vec

In [ ]:
assert get_sent_vec(sent_toks, lex).shape[0] == 768

NameError: name 'sent_toks' is not defined

In [ ]:
def get_cos_sim(vec_1, vec_2):
    cos_sim = CosineSimilarity(dim=0)
    sim_vecs = cos_sim(vec_1, vec_2)
    return float(sim_vecs)

In [ ]:
def get_sent_sim(sent_1, sent_2, lex):
    sent_vec_1 = get_sent_vec_
    cos_sim = get_cos_sim(sent_vec_1, sent_vec_2)

## Type-based embeddings

In [ ]:
embedding = WordEmbeddings('glove')

In [ ]:
sentence = Sentence('I can can a can.')
embedding.embed(sentence)

In [ ]:
for token in sentence:
    print(token)
    print(token.embedding.shape)

## Token-based embeddings

### Toy example: 'bank'

In [ ]:
sents = [proc_sentence(sent) for sent in sents]

In [ ]:
sent_vecs = [get_sent_vec(sent, lex) for sent in sents]

In [ ]:
sent_vecs[0].shape

In [ ]:
len(sent_vecs)

In [ ]:
get_cos_sim(sent_)

In [ ]:
print(
    cos(vecs[0], vecs[1]),
    cos(vecs[0], vecs[2]),
    cos(vecs[1], vecs[2]),    
       )

In [ ]:
assert cos(vecs[0], vecs[1]) > cos(vecs[0], vecs[2])
assert cos(vecs[0], vecs[1]) > cos(vecs[1], vecs[2])

### 'Cell'

In [ ]:
lex = 'cell'

In [ ]:
comments = [
    'I bought a new cell phone yesterday.',
    'Stem cell research has found a cure for cancer.'
]

In [ ]:
sents = [Sentence(comm) for comm in comments]

In [ ]:
sents_embed = embedding.embed([sent for sent in sents])

In [ ]:
vecs = [tok.embedding for tok in toks]

In [ ]:
for tok in Sentence(comments[0]):
    if tok.text == 'cell':
        print(tok, tok.embedding)

### 'Anglo-Saxon' 

In [ ]:
from flair.data import Sentence

In [ ]:
from flair.embeddings import TransformerWordEmbeddings

In [ ]:
from random import choices

In [ ]:
txt = df.body[0]

In [ ]:
txt = "This is a sentence. This is another sentence. I love Berlin."

In [ ]:
splitter = SegtokSentenceSplitter()

In [ ]:
sents = splitter.split(txt)

print(len(sents), '\n')

for sent in sents:
    print(len(sent))

In [ ]:
random.choices(sents, k=3)

In [ ]:
random.choices(df['body'], k=3)

In [ ]:
comments = [
    'At the same time, as the Anglo Saxons began settling in Roman territory, they began adopting vulgar Latin words into their vocabulary.',
    'Islamophobia here tends to refer to the racism from the WASP (white Anglo-Saxon Protestant)-types.'
]

In [ ]:
import re

rep = {"anglo saxon": "anglo-saxon", "anglo-saxons": "anglo-saxon"} 
rep = dict((re.escape(k), v) for k, v in rep.items())
pattern = re.compile("|".join(rep.keys()))


In [ ]:
[pattern.sub(lambda m: rep[re.escape(m.group(0))], comment) for comment in comments]

In [ ]:
comments_sents = [splitter.split(comment) for comment in comments]

print(len(comments_sents), '\n')

for comment in comments_sents:
    print(len(comment))
    for sent in comment:
        print(sent)


In [ ]:
sent.replace()

In [ ]:
comments_low = [comm.lower() for comm in comments]

In [ ]:
lex = 'Anglo-Saxon'
patterns = ['Anglo-Saxon', 'Anglo Sa']

comments_tokmerge

In [ ]:
sents = [Sentence(txt) for txt in txts]

In [ ]:
sents_toks = []
for sent in sents:
    toks = 0
    for tok in sent:
        if tok.text == 'Anglo-Saxon':
            toks += 1
    if toks >= 1:
        print(sent)

In [ ]:
random.choices(sents, k=3)

In [ ]:
sent = 'Anglo-Saxon Anglo Saxon people'
sent = Sentence(sent)

for tok in sent:
    if tok.text == 'Anglo-Saxon':
        print(tok)